 ### 01 Actividad práctica: Evaluación realista en el Titanic: Riesgos del sobreajuste y la evaluación poco realista
Objetivo:
Los estudiantes aprenderán cómo un modelo de Machine Learning puede tener un rendimiento sobreestimado cuando el conjunto de prueba no es independiente y cómo este tipo de error puede afectar la toma de decisiones en proyectos reales.
Requisitos:
● Python (con bibliotecas como pandas, scikit-learn, matplotlib)
● Jupyter Notebook o entorno similar
● Conjunto de datos de Titanic disponible en Kaggle Titanic Dataset.
Descripción de la actividad:
1. Cargar y preprocesar los datos: Cargar el conjunto de datos Titanic desde un archivo CSV o Kaggle, preprocesar los datos (tratamiento de valores faltantes, codificación de variables categóricas, etc.).
2. Entrenar un modelo: Usar un modelo de clasificación (por ejemplo, Árbol de Decisión o Regresión Logística) para predecir la supervivencia de los pasajeros basándose en características como la clase, el sexo, la edad, el puerto de embarque, etc.
3. Evaluación del modelo:
○ Evaluación inicial (métrica con datos de prueba no independientes): Evaluar el modelo usando un conjunto de prueba que tenga algunos datos duplicados o filtrados del conjunto de entrenamiento.
○ Evaluación con datos de prueba verdaderamente independientes: Evaluar el modelo en un conjunto de prueba completamente independiente (datos que no se han utilizado en el entrenamiento).
4. Observación de las métricas: Los estudiantes compararán las métricas de rendimiento (precisión, recall, F1-score, etc.) obtenidas de ambas evaluaciones (no independiente e independiente) y observarán cómo se inflan las métricas en el primer caso.
5. Conclusión: Los estudiantes documentarán cómo un conjunto de prueba no independiente puede dar lugar a una evaluación sobreestimada y cómo esto puede afectar la implementación de modelos en la práctica.

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [34]:
df = pd.read_csv('titanic.csv')
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocesamiento de los datos


In [35]:

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer #Ayuda  a rellenar valores faltantes
from sklearn.compose import ColumnTransformer #transforma las columnas categoricas en variables
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



In [36]:
# Definir las columnas numéricas y 
numeric_features = ['Age','Fare']
categorical_features = ['Embarked','Sex']


#Verificar si las columnas categoricas tienen valores nulos
non_empty_categorical = [col for col in categorical_features if df[col].notna().any()]


#Crear transformador para columnas numéricas
numeric_transformer = SimpleImputer(strategy='mean')


#Crear transformador para columnas categóricas  
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
])


#Combinar los transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, non_empty_categorical)
    ]
)


#Seleccionamos las cracateristicas(x) y la variable objetivo (y)
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = df['Survived'] #Lo que queremos predecir


#Aplicar el preprocesamiento a las caracteristicas

X = preprocessor.fit_transform(X)


#Convertir el array resultante a un Dataframe
X = pd.DataFrame(X)


## Dividir los datos del conjunto de entrenamiento y prueba

In [37]:
test_size = 0.2
random_state = 42
shuffle = True #Baraja los datos antes de dividir
stratify = 'mean'
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=test_size, random_state=random_state, shuffle=shuffle, stratify = y)


#Introducir un conjunto de datos de prieba no independiente
X_test_fake = pd.concat ([X_test, X_train.sample(10)])
y_test_fake = pd.concat ([y_test, y_train.sample(10)])



## Entrenar el modelo de clasificación

In [38]:
#Entrenar el modelo de árbol de decisión con hiperparámetros especializados
model= DecisionTreeClassifier(max_depth=5, min_samples_split=5, random_state=random_state, min_samples_leaf=2)
model.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

## Evaluar el modelo en el conjunto de prueba no independiente

In [39]:
y_pred_fake = model.predict(X_test_fake)


#Imprimir las métricas de envaliación
print('Evaluación con un conjunto de prueba no independiente')
print('Accuracy:', accuracy_score(y_test_fake, y_pred_fake))
print('Matriz de confusion:\n', confusion_matrix(y_test_fake, y_pred_fake))
print('Reporte de clasificación:\n', classification_report(y_test_fake, y_pred_fake))


Evaluación con un conjunto de prueba no independiente
Accuracy: 0.798941798941799
Matriz de confusion:
 [[103  17]
 [ 21  48]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.83      0.86      0.84       120
           1       0.74      0.70      0.72        69

    accuracy                           0.80       189
   macro avg       0.78      0.78      0.78       189
weighted avg       0.80      0.80      0.80       189



## Evaluar el modelo con el conjunto de prueba independiente

In [40]:
y_pred= model.predict(X_test)
print('Evaluación con un conjunto de prueba independiente')
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Matriz de confusion:\n', confusion_matrix(y_test, y_pred))
print('Reporte de clasificación:\n', classification_report(y_test, y_pred))

Evaluación con un conjunto de prueba independiente
Accuracy: 0.8044692737430168
Matriz de confusion:
 [[96 14]
 [21 48]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.82      0.87      0.85       110
           1       0.77      0.70      0.73        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.80      0.80      0.80       179

